In [3]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import folium
import openrouteservice as ors
from geopy.distance import great_circle
import webbrowser
from dotenv import load_dotenv

In [2]:
def get_token():
    load_dotenv('./.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    client_id = os.environ.get("X-ClientId")
    pass_key = os.environ.get("passKey")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [3]:
get_token()

b'{"code": "01", "description": "Token extend  into control-cache Data recovered  OK, (lapsed: 137 millsecs)", "datetime": "2024-02-16T14:33:30.977078", "data": [{"XClientId": "09141f1a-bf02-4e23-80ee-e7fa7ea819d2", "nameApp": "OPENAPI MobilityLabs", "updatedAt": "2024-02-02T14:01:48.5770000", "userName": "silviluliuma", "lastUpdate": {"$date": 1708090364012}, "idUser": "26ddfea8-f2ce-438f-89e4-5110c9abd07e", "priv": "U", "tokenSecExpiration": 86399, "email": "valeromsilvia@gmail.com", "tokenDteExpiration": {"$date": 1708180363012}, "flagAdvise": true, "accessToken": "7673865f-2b86-4f37-957d-9ac06cf5b7c0", "apiCounter": {"current": 2, "dailyUse": 20000, "owner": 0, "licenceUse": "Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!", "aboutUses": "If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylabs.emtmadrid.es/doc/new-app an

In [4]:
def get_stations():
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    print(token)
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [5]:
get_stations()

7673865f-2b86-4f37-957d-9ac06cf5b7c0


{'code': '00',
 'description': '613 bases recovered',
 'datetime': '2024-02-16T14:33:32.622583',
 'data': [{'activate': 1,
   'address': 'Calle María de las Mercedes de Borbón, 214,',
   'code_district': '16',
   'code_suburb': '166',
   'dock_bikes': 5,
   'free_bases': 18,
   'geofence': {'type': 'Polygon',
    'coordinates': [[[-3.620703128, 40.493318578],
      [-3.620703128, 40.492418022],
      [-3.619524872, 40.492418022],
      [-3.619524872, 40.493318578],
      [-3.620703128, 40.493318578]]]},
   'geofenced_capacity': 20,
   'geometry': {'type': 'Point', 'coordinates': [-3.620114, 40.4928683]},
   'id': 2365,
   'integrator': '3c34ea72-2a03-4a3c-be97-cf32be4b0ebe',
   'light': 0,
   'name': '262 - María de las Mercedes de Borbón, 214',
   'no_available': 0,
   'number': '262',
   'reservations_count': 0,
   'tipo_estacionPBSC': 'FIXED',
   'total_bases': 23,
   'virtualDelete': False,
   'virtual_bikes': [],
   'virtual_bikes_num': 0,
   'bikesGo': []},
  {'activate': 1,
   '

In [10]:
stations = get_stations()
stations_real_time = pd.DataFrame(stations["data"])


d1b5b864-e275-4dab-b8af-8c916182ddcd


In [11]:
stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
stations_real_time = stations_real_time.drop(["geofence", "activate", "geometry", "integrator", "reservations_count", "no_available", "tipo_estacionPBSC", "virtualDelete", "virtual_bikes", "virtual_bikes_num", "code_suburb", "geofenced_capacity", "bikesGo"], axis=1)
stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
stations_real_time

,address,code_district,dock_bikes,free_bases,id,light,name,number,total_bases,longitude,latitude,coordinates
0,"Calle María de las Mercedes de Borbón, 214,",16,11,12,2365,2,"262 - María de las Mercedes de Borbón, 214",262,23,-3.620114,40.492868,"(-3.620114, 40.4928683)"
1,"Calle María de las Mercedes de Borbón, 98,",16,5,18,2366,0,"611 - María de las Mercedes de Borbón, 98",611,23,-3.612107,40.490908,"(-3.6121068531215528, 40.490908297996654)"
2,"237- calle Francisco Balseiro,Comunidad de Mad...",06,14,10,2362,2,237 - calle Francisco Balseiro,237,24,-3.710274,40.451682,"(-3.7102736, 40.4516819)"
3,"para borrar,",16,0,0,1404,3,0 - para borrar,0,0,-3.620180,40.492910,"(-3.62018, 40.49291)"
4,"Calle Miguel Moya nº 1,",01,21,6,1406,1,2 - Miguel Moya,2,27,-3.705690,40.420400,"(-3.70569, 40.4204)"
...,...,...,...,...,...,...,...,...,...,...,...,...
607,"Av Abrantes, 55,",11,7,11,2153,2,"377 - Av Abrantes, 55",377,18,-3.727836,40.380918,"(-3.72783615, 40.38091762)"
608,"Calle de Mota del Cuervo, 70,",16,12,12,2154,2,"539 - Calle de Mota del Cuervo, 70",539,24,-3.631357,40.464876,"(-3.63135715, 40.4648762)"
609,"Calle Seseña, 93,",10,18,6,2155,1,"334 - Seseña, 93",334,24,-3.768156,40.394973,"(-3.76815631, 40.39497294)"
610,"Calle Cardaño, 10,",10,10,14,2156,2,"335 - Calle Cardaño, 10",335,24,-3.770575,40.396598,"(-3.77057541, 40.39659757)"


In [12]:
def get_district(df, district_number):
    result = df[df["code_district"] == district_number]
    return result

district_03 = get_district(stations_real_time, "03")
district_03

,address,code_district,dock_bikes,free_bases,id,light,name,number,total_bases,longitude,latitude,coordinates
38,"Calle Felipe IV, 3,",03,13,10,2321,2,"283 - Calle Felipe IV, 3",283,23,-3.691906,40.415208,"(-3.6919057, 40.4152082)"
93,"Plaza de la Independencia nº 6,",03,18,5,1464,1,60 - Plaza de la Independencia,60,23,-3.688398,40.419752,"(-3.688398, 40.419752)"
94,"Calle O'Donnell nº 28,",03,20,7,1465,1,61 - Narváez,61,27,-3.675314,40.421455,"(-3.6753139, 40.4214551)"
95,"Calle O'Donnell nº 50,",03,20,7,1466,1,62 - O'Donnell,62,27,-3.672497,40.421315,"(-3.6724968, 40.4213148)"
96,"Calle Ibiza nº 62,",03,14,9,1467,2,63 - Ibiza,63,23,-3.670896,40.417924,"(-3.6708959, 40.4179237)"
97,"Calle Antonio Maura nº 15,",03,17,7,1468,1,65 - Antonio Maura,65,24,-3.689425,40.416701,"(-3.689424797942769, 40.4167014469994)"
99,"Calle Espalter nº 1,",03,8,11,1470,2,68 - Espalter,68,19,-3.691323,40.412769,"(-3.6913232, 40.4127692)"
100,"Avenida de Alfonso XII nº 54,",03,13,14,1471,2,69 - Puerta del Ángel Caído,69,27,-3.688822,40.409808,"(-3.688822, 40.409808)"
101,"Avenida de Menéndez Pelayo nº 63,",03,21,6,1472,1,70 - Puerta del Doce de Octubre,70,27,-3.677873,40.415382,"(-3.677872542327878, 40.415382385574716)"
102,"Calle Doce de Octubre nº 28,",03,11,12,1473,2,71 - Doce de Octubre,71,23,-3.673900,40.416100,"(-3.6739, 40.4161)"


In [13]:
district_01 = stations_real_time[stations_real_time["code_district"]== "01"]
district_02 = stations_real_time[stations_real_time["code_district"]== "02"]
district_03 = stations_real_time[stations_real_time["code_district"]== "03"]
district_04 = stations_real_time[stations_real_time["code_district"]== "04"]
district_05 = stations_real_time[stations_real_time["code_district"]== "05"]
district_06 = stations_real_time[stations_real_time["code_district"]== "06"]
district_07 = stations_real_time[stations_real_time["code_district"]== "07"]
district_08 = stations_real_time[stations_real_time["code_district"]== "08"]
district_09 = stations_real_time[stations_real_time["code_district"]== "09"]
district_10 = stations_real_time[stations_real_time["code_district"]== "10"]
district_11 = stations_real_time[stations_real_time["code_district"]== "11"]
district_12 = stations_real_time[stations_real_time["code_district"]== "12"]
district_13 = stations_real_time[stations_real_time["code_district"]== "13"]
district_14 = stations_real_time[stations_real_time["code_district"]== "14"]
district_15 = stations_real_time[stations_real_time["code_district"]== "15"]
district_16 = stations_real_time[stations_real_time["code_district"]== "16"]
district_17 = stations_real_time[stations_real_time["code_district"]== "17"]
district_18 = stations_real_time[stations_real_time["code_district"]== "18"]
district_19 = stations_real_time[stations_real_time["code_district"]== "19"]
district_20 = stations_real_time[stations_real_time["code_district"]== "20"]
district_21 = stations_real_time[stations_real_time["code_district"]== "21"]

In [14]:
district_02

,address,code_district,dock_bikes,free_bases,id,light,name,number,total_bases,longitude,latitude,coordinates
48,"279 - Paseo de los Melancólicos, 1,",02,19,4,2331,1,"279 - Paseo de los Melancólicos, 1",279,23,-3.718482,40.410445,"(-3.7184816, 40.4104448)"
50,"Paseo de las Yeserías, 15,",02,10,13,2333,2,"276 - Paseo de las Yeserías, 15",276,23,-3.709917,40.399005,"(-3.7099166, 40.39900539)"
79,"Calle Ribera de Curtidores nº 28,",02,17,10,1450,2,46 - Ribera de Curtidores,46,27,-3.707126,40.405315,"(-3.7071259, 40.4053153)"
80,"Glorieta de Embajadores nº 6,",02,17,7,1451,1,47 - Embajadores 1,47,24,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
159,"PASEO DE LOS MELANCÓLICOS 73,",02,16,7,2247,2,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,-3.718569,40.401782,"(-3.718569, 40.401782)"
175,"271 - Calle Amaltea , 1,",02,2,22,2265,0,"271 - Calle Amaltea , 1",271,24,-3.683170,40.394706,"(-3.68317, 40.394706)"
177,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",02,13,10,2267,2,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,23,-3.708212,40.402054,"(-3.7082123, 40.4020545)"
179,"Calle Méndez Alvaro, 44,",02,11,12,2269,2,"274 - Méndez Alvaro, 73",274,23,-3.681343,40.396758,"(-3.68134346, 40.39675755)"
205,"Paseo de los Pontones, 45,",02,13,10,2299,2,"278 - Paseo de los Pontones, 45",278,23,-3.719600,40.404043,"(-3.7196, 40.4040433)"
210,"Paseo virgen de puerto, 25,",02,10,5,2304,2,"277 - Paseo Virgen del Puerto, 25",277,15,-3.721178,40.411622,"(-3.721178, 40.411622)"


In [15]:
def get_light1(df):
    df_light1 = df[df["light"]==1]
    return df_light1

In [16]:
get_light1(district_02)

,address,code_district,dock_bikes,free_bases,id,light,name,number,total_bases,longitude,latitude,coordinates
48,"279 - Paseo de los Melancólicos, 1,",02,19,4,2331,1,"279 - Paseo de los Melancólicos, 1",279,23,-3.718482,40.410445,"(-3.7184816, 40.4104448)"
80,"Glorieta de Embajadores nº 6,",02,17,7,1451,1,47 - Embajadores 1,47,24,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
232,"Paseo de Santa María de la Cabeza nº 58,",02,25,1,1527,1,121 - Santa María de la Cabeza,121,27,-3.698603,40.401749,"(-3.6986027, 40.4017487)"
293,"Paseo de la Chopera,33,Comunidad de Madrid España",02,17,6,2205,1,"267 - Paseo de la Chopera, 33",267,23,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
311,"Paseo de la Esperanza nº 2,",02,15,4,1569,1,163 - Paseo de la Esperanza,163,19,-3.706445,40.403759,"(-3.706445, 40.4037586)"
319,"Calle Embajadores nº 97,",02,23,4,1577,1,171 - Embajadores-Cáceres,171,27,-3.698402,40.399701,"(-3.6984023, 40.3997013)"
443,"Calle Puerto Serrano, 1,",02,18,5,1986,1,"265 - Calle Puerto Serrano, 1",265,23,-3.688461,40.385412,"(-3.688461, 40.385412)"


In [17]:
def get_light0(df):
    df_light0 = df[df["light"]==0]
    return df_light0

In [18]:
get_light0(district_02)

,address,code_district,dock_bikes,free_bases,id,light,name,number,total_bases,longitude,latitude,coordinates
175,"271 - Calle Amaltea , 1,",02,2,22,2265,0,"271 - Calle Amaltea , 1",271,24,-3.683170,40.394706,"(-3.68317, 40.394706)"
315,"Calle Segovia nº 45,",02,8,19,1573,0,167 - Segovia 45,167,27,-3.717489,40.413748,"(-3.71748917032373, 40.41374770742206)"
317,"Calle Toledo nº 181,",02,7,17,1575,0,169 - Pirámides,169,24,-3.713823,40.401589,"(-3.713823, 40.401589)"
323,"Calle Jaime el Conquistador nº 30,",02,6,13,1581,0,175 - Jaime el Conquistador,175,19,-3.698345,40.396226,"(-3.69834452, 40.3962264)"
324,"Calle Bolívar nº 3,",02,3,20,1583,0,177 - Legazpi,177,23,-3.694188,40.391447,"(-3.6941876374343896, 40.391447085678806)"
327,"Calle Retama nº 5,",02,6,17,1586,0,180 - Méndez Álvaro,180,23,-3.675890,40.394042,"(-3.67589, 40.3940423)"
382,"Calle Sodio nº 1B,",02,7,16,1641,0,235 - Embajadores 191,235,23,-3.691076,40.389041,"(-3.691076, 40.389041)"
473,"Calle del Cobre (Metro Arganzuela Planetario),",02,3,20,2017,0,268 - Calle del Cobre (M. Planetario),268,23,-3.689031,40.393383,"(-3.689031463575172, 40.39338327980599)"
518,"Calle Ariel, 7,",02,2,22,2064,0,"269 - Calle Ariel, 7",269,24,-3.679702,40.393024,"(-3.67970176633988, 40.393023802369754)"


In [19]:
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, vehicle_start).meters)
    return nearest_station

In [20]:
stations_real_time

,address,code_district,dock_bikes,free_bases,id,light,name,number,total_bases,longitude,latitude,coordinates
0,"Calle María de las Mercedes de Borbón, 214,",16,11,12,2365,2,"262 - María de las Mercedes de Borbón, 214",262,23,-3.620114,40.492868,"(-3.620114, 40.4928683)"
1,"Calle María de las Mercedes de Borbón, 98,",16,5,18,2366,0,"611 - María de las Mercedes de Borbón, 98",611,23,-3.612107,40.490908,"(-3.6121068531215528, 40.490908297996654)"
2,"237- calle Francisco Balseiro,Comunidad de Mad...",06,14,10,2362,2,237 - calle Francisco Balseiro,237,24,-3.710274,40.451682,"(-3.7102736, 40.4516819)"
3,"para borrar,",16,0,0,1404,3,0 - para borrar,0,0,-3.620180,40.492910,"(-3.62018, 40.49291)"
4,"Calle Miguel Moya nº 1,",01,21,6,1406,1,2 - Miguel Moya,2,27,-3.705690,40.420400,"(-3.70569, 40.4204)"
...,...,...,...,...,...,...,...,...,...,...,...,...
607,"Av Abrantes, 55,",11,7,11,2153,2,"377 - Av Abrantes, 55",377,18,-3.727836,40.380918,"(-3.72783615, 40.38091762)"
608,"Calle de Mota del Cuervo, 70,",16,12,12,2154,2,"539 - Calle de Mota del Cuervo, 70",539,24,-3.631357,40.464876,"(-3.63135715, 40.4648762)"
609,"Calle Seseña, 93,",10,18,6,2155,1,"334 - Seseña, 93",334,24,-3.768156,40.394973,"(-3.76815631, 40.39497294)"
610,"Calle Cardaño, 10,",10,10,14,2156,2,"335 - Calle Cardaño, 10",335,24,-3.770575,40.396598,"(-3.77057541, 40.39659757)"


In [30]:
def get_user_location():
    load_dotenv('./.env')
    key = os.environ.get("api_location_key")
    print(key)
    response = requests.get("https://api.ipgeolocation.io/ipgeo?apiKey=" + key)
    user_coords = float(response.json()["latitude"]), float(response.json()["longitude"])
    return user_coords

In [32]:
user_coords = [get_user_location()[0], get_user_location()[1]]
user_coords

cb73229833f14891b149e648936ab5e3
cb73229833f14891b149e648936ab5e3


[42.22986, -8.75608]

In [26]:
load_dotenv('../.env')
client = ors.Client(key=os.environ.get("openroute_api_key"))

# Function to find the nearest station to given coordinates
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, coords).meters)
    return nearest_station

# Function to create a route between two sets of coordinates
def create_route(client, start_coords, end_coords):
    return client.directions(
        coordinates=[start_coords, end_coords],
        profile='driving-car',
        format='geojson'
    )

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

# Creo un mapa con Folium
m = folium.Map(location=[vehicle_start[1], vehicle_start[0]], zoom_start=12)

# Añado marcador morado para la central (vehicle_start)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='CENTRAL EMT', icon=folium.Icon(color='purple')).add_to(m)

distrito02_low = get_light0(district_02).copy()
distrito02_high = get_light1(district_02).copy()

distrito02_high['visited'] = False
distrito02_low['visited'] = False

# Inicio de la ruta
current_coords = vehicle_start
van = "empty"

coords_list = [current_coords]

stop_counter = 1

for i in range(100):
    if van == "empty":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)].empty:
            nearest_station = find_nearest_to_coords(distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'visited'] = True
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "full"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with high occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='orange')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
    elif van == "full":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)].empty:
            nearest_station = find_nearest_to_coords(distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'visited'] = True
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "empty"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with low occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='green')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
final_route = create_route(client, coords_list[-1], vehicle_start)
folium.PolyLine(locations=[coord[::-1] for coord in final_route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)

m

NameError: name 'van_sidebar' is not defined